# Parser de Relatório MetLife Sintético

Adapte o caminho do PDF conforme necessário, ou faça upload pelo Colab. O script gera um arquivo Excel com os repasses e GTOs extraídos.

In [ ]:
!pip install pdfplumber openpyxl

## Upload do PDF

In [ ]:
# Se estiver usando o Colab, faça upload do PDF e pegue o caminho
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]  # Nome do arquivo enviado

In [ ]:
import pdfplumber  # Biblioteca para ler e extrair texto de arquivos PDF
import pandas as pd  # Biblioteca para manipulação de dados em tabelas (DataFrames)
import re  # Biblioteca para trabalhar com expressões regulares (padrões de texto)
from google.colab import files

def extrair_dados_metlife_s(pdf_path):
    resultados = []
    repasses = []

    texto_completo = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            texto = page.extract_text()
            if texto:
                texto_completo += texto + "\n"

    linhas = texto_completo.split('\n')

    def to_float(valor):
        try:
            return float(valor.replace('.', '').replace(',', '.').strip())
        except:
            return 0.0

    i = 0
    while i < len(linhas):
        if linhas[i].strip().startswith("PRESTADOR:"):
            repasse_match = re.search(r"REPASSE:\s*(\d{7})", linhas[i].strip())
            num_repasse = repasse_match.group(1) if repasse_match else ""
            if num_repasse:
                repasses.append({"Repasses": num_repasse})
        i += 1

    i = 0
    while i < len(linhas):
        # Initialize variables at the beginning of each iteration
        gto = ""
        glosa = ""
        valor_info = valor_glosa = valor_pago = 0.0
        situação_gto = "P"
        cod_paciente = ""
        nome_social = ""


        if linhas[i].strip().startswith("GTO:"):
            gto_match = re.search(r"GTO:\s*(\d{9})", linhas[i].strip())
            gto = gto_match.group(1) if gto_match else ""

            if i + 1 < len(linhas):
                linha_paciente = linhas[i + 1].strip()
                paciente_match = re.search(r"CÓDIGO/NOME BENEFICIÁRIO:\s*(\d{14})", linha_paciente)
                cod_paciente = paciente_match.group(1) if paciente_match else ""
                if cod_paciente:
                    tokens = linha_paciente.split()
                    try:
                        idx_ini = tokens.index("-") + 1
                        idx_fim = tokens.index("TITULAR:")
                        nome_tokens = tokens[idx_ini:idx_fim]
                        nome_social = " ".join(nome_tokens)
                    except ValueError:
                        pass
        if linhas[i].strip().startswith("TOTAL DA GTO"):
            linha_valores = linhas[i].strip()
            if linha_valores:
                tokens = linha_valores.split()
                try:
                    valor_glosa = to_float(tokens[3])
                    valor_pago = to_float(tokens[4])
                    valor_info = valor_glosa + valor_pago
                    if valor_glosa > 0:
                        situação_gto = "GL"
                except ValueError:
                    pass
        if valor_info > 0:
            resultados.append({
                "Data": "",
                "Mariana": "",
                "Repetida?": "",
                "GTO": gto,
                "Plano": "MetLife",
                "Situação": situação_gto,
                "Cod Glosa": "",
                "Detalhes": glosa,
                "Valor GTO": valor_info,
                "Valor Glosa": valor_glosa,
                "Valor Final": valor_pago,
                "Localidade": "",
                "Acesso": "",
                "Paciente": nome_social,
                "ID": cod_paciente,
                "Procedimento": "",
                "Index": "Metlife." + gto,
            })
        i += 1

    df_repasse = pd.DataFrame(repasses)
    df_gto = pd.DataFrame(resultados)
    saida = "procedimentos_metlife_sintetico.xlsx"
    with pd.ExcelWriter(saida, engine='openpyxl') as writer:
        df_repasse.to_excel(writer, index=False)
        df_gto.to_excel(writer, index=False, startrow=len(df_repasse) + 2)
    print(f"Arquivo exportado com sucesso: {saida}")
    return saida


## Executando o parser

In [ ]:
saida = extrair_dados_metlife_s(pdf_path)

## Download do resultado

In [ ]:
files.download(saida)